In [ ]:
%load_ext watermark


In [ ]:
from downstream import dstream
from hstrat import hstrat
from hstrat import _auxiliary_lib as hstrat_aux
import pandas as pd
import scipy
import seaborn as sns
import numpy as np
import typing

In [ ]:
%watermark -diwmuv -iv


In [ ]:
teeplot_subdir = "2025-09-07-shortcut-naive-comparison"
teeplot_subdir


## Prep Data


In [ ]:
reconstruction_error_data = pd.read_csv("https://osf.io/7j3t4/download")
reconstruction_error_data.head()


### Power Analysis

Assuming the null hypothesis for a given sample size and significance level, finds the lowest number $x$ such that $$P(\text{positives} \le x) \ge 0.95$$ Hypothetically, this means that, given a certain effect size, the effect size is reliably detected if the number of positives is greater than this number.

In [ ]:
def compute_expected_positive_rate(num_trials: int, significance: float) -> int:
    for i in range(num_trials + 1):
        if scipy.stats.binom.cdf(i, num_trials, significance) >= 1 - significance:
            return i 
compute_expected_positive_rate(40, 0.05)

#### Determining the effect size

In [ ]:
def min_detected_effect_size(significance: float, *, num_tests: int = 10, sample_size: typing.Optional[int] = None) -> float:
    for multiplier in np.linspace(0, 1, 51):
        positives = 0
        for _ in range(num_tests):
            groups = reconstruction_error_data.groupby(["differentia_bitwidth", "surface_size", "fossil_interval"])
            for _, df in groups:
                example_data = df[df["reconstruction_algorithm"] == "shortcut"]["error_dropped_fossils"].to_numpy()
                if sample_size is not None:
                    num_repeats = int(np.ceil(sample_size / example_data.size))
                    example_data = example_data.repeat(num_repeats)[:sample_size]
                _, p = scipy.stats.mannwhitneyu(
                    example_data,
                    example_data + np.random.uniform(size=example_data.size) * example_data.mean() * multiplier,
                    alternative="two-sided",
                )

                if p < significance:
                    positives += 1
        if positives / num_tests > compute_expected_positive_rate(len(groups), significance):
            return {
                "effect_size": float(multiplier),
                "sample_size": example_data.size
            }
        
min_detected_effect_size(0.05, sample_size=40)

### Running statistical test

From the power analysis, we know that if there are more than 4 positives then we have a significant difference. We also know that this is capable of detecting a difference of around 0.40 (with a sample size of 40).

In [ ]:
for info, df in reconstruction_error_data.groupby(["differentia_bitwidth", "surface_size", "fossil_interval"]):
    stat, p = scipy.stats.mannwhitneyu(
        df[df["reconstruction_algorithm"] == "shortcut"]["error_dropped_fossils"],
        df[df["reconstruction_algorithm"] == "naive"]["error_dropped_fossils"],
        alternative="two-sided",
    )

    if p < 0.05:
        print(
            f"Significant difference in reconstruction error between shortcut and naive for differentia_bitwidth={info[0]}, surface_size={info[1]}, fossil_interval={info[2]}"
        )
        sns.violinplot(
            data=df,
            x="reconstruction_algorithm",
            y="error_dropped_fossils",
            inner="point",
            split=False,
        ).set_title(
            f"differentia_bitwidth={info[0]}, surface_size={info[1]}, fossil_interval={info[2]}"
        )